General information like installation instructions in this supplementary notebook "Project-1_Code-Walkthrough". Please take a glance at this notebook completely before you start this assignment.

### Installation instructions
Use Python 3.5+.
You'll need jax, EVO, g2o for this project. Don't worry, I have provided sufficient details below for you to get started quickly. Note that the instructions are for Linux and if you are using any other OS, you'll have to go through their respective webpages.

If you face any issues related to coding/installation, please raise an [issue here](https://github.com/Shubodh/MR-project1-pgo/issues). For any conceptual doubts, you can ask on Moodle or Teams as usual.

## 1. Installing and using JAX

JAX is an auto-differentiation library for native Python and Numpy code which does gradient-based optimization. Auto-differentiation forms the backbone of deep learning libraries like PyTorch.

Activate your standard environment from Assignment-3. Then   
```
pip install --upgrade pip     
pip install --upgrade jax jaxlib 
```

[See this](https://github.com/google/jax#installation) for more information. (CPU version should be enough for this project.)

In [1]:
import jax.numpy as jnp
"""
Basically, use "jnp" instead of using "np", our favourite numpy library. 
All functions work as it is (at least that are required for this project).
Be careful though:
JAX works on python functions that are "functionally pure": 
For the sake of our project, that just means using array datatype everywhere 
(or 'jnp.array()' in particular) instead of using other datatype, say lists for
storing arrays or matrices. Whenever you face some datatype issue with jax, 
first try to convert it to jax numpy array using `jnp.array()`.

Tip: In my experience, jnp's errors didn't seem very readable as compared to np.
So use "np" first for most of the code and the moment the necessity for "jnp" starts, 
replace all np's with jnp's. Directly replacing should work fine. This is only a tip for easier 
debugging. Please note when you're submitting, there should be only `jax.numpy`.

"""
from jax import jacfwd

In [2]:
# Define some simple function.
def sigmoid(x):
    return 0.5 * (jnp.tanh(x / 2) + 1)
# Note that here, I want a derivative of a "vector" output function (inputs*a + b is a vector) wrt a input 
# "vector" a at a0: Derivative of vector wrt another vector is a matrix: The Jacobian
def simpleJ(a, b, inputs): #inputs is a matrix, a & b are vectors
    return sigmoid(jnp.dot(inputs, a) + b)

inputs = jnp.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])

b = jnp.array([0.2, 0.1, 0.3, 0.2])
a0 = jnp.array([0.1,0.7,0.7])

# Isolate the function: variables to be differentiated from the constant parameters
f = lambda a: simpleJ(a, b, inputs) # Now f is just a function of variable to be differentiated

J = jacfwd(f)
# Till now I have only calculated the derivative, it still needs to be evaluated at a0.
J(a0)

/home/gera/anaconda3/envs/mr_assignment1/lib/python3.7/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


DeviceArray([[ 0.07388726,  0.1591418 ,  0.10940998],
             [ 0.20861849, -0.2560318 ,  0.03555997],
             [ 0.12171669,  0.01404423, -0.3042917 ],
             [ 0.17407253, -0.58573055,  0.3269741 ]], dtype=float32)

## 2. Using EVO
[EVO](https://github.com/MichaelGrupp/evo/) is a SLAM evaluation based library for handling, evaluating and comparing the trajectory output of odometry and SLAM algorithms.

```
pip3 install evo --upgrade --no-binary evo

```
See [EVO](https://github.com/MichaelGrupp/evo/) for more info. EVO doesn't directly support `g2o` format. We have to first convert `g2o` to `kitti`. A small script named `g2o_to_kitti.py` which does exactly this has been provided in `misc` folder. `python input_file.g2o name_of_output_file.kitti` to convert it to `kitti` format. Ex: `python gt.g2o gt.kitti`.

Now that it is in `kitti` format, run
```
evo_rpe kitti gt.kitti opt.kitti -v --plot --plot_mode xy
evo_ape kitti gt.kitti opt.kitti -v --plot --plot_mode xy
evo_traj kitti gt.kitti opt.kitti -v --plot --plot_mode xy
```
Using EVO, the bare minimum you have to report is `mean absolute pose error (ape)` and `mean relative pose error (rpe)`. However, you are encouraged to use tools like `evo_traj` and [more](https://github.com/MichaelGrupp/evo/#command-line-interface). Marks will be awarded based on overall analysis & presentation which would reflect your understanding.
Note that `EVO` and `g2o_viewer` could help you in debugging.

## 3. Setting up g2o
g2o is an open-source C++ framework for optimizing graph-based nonlinear error functions. We will be using g2o's binaries in this project.

More regarding g2o from their homepage:
>A wide range of problems in robotics as well as in computer-vision involve the minimization of a non-linear error function that can be represented as a graph. Typical instances are simultaneous localization and mapping (SLAM) or bundle adjustment (BA). The overall goal in these problems is to find the configuration of parameters or state variables that maximally explain a set of measurements affected by Gaussian noise. g2o is an open-source C++ framework for such nonlinear least squares problems. 

Note that the instructions are for Linux and if you are using any other OS, you'll have to go through the official pages.

Please go through [the official page first](https://github.com/RainerKuemmerle/g2o). If you are stuck, only then refer to the following points. Please install the requirements carefully before actual compilation.

### Requirements
As written [there](https://github.com/RainerKuemmerle/g2o#requirements), cmake & libeigen3-dev.

Both of them can be installed via apt-get. Some useful commands:

```
sudo apt-get -y install cmake
which cmake #check the installation
cmake --version
sudo apt-get install libeigen3-dev
```

`libeigen3-dev` will reside in `usr/include/eigen3/` if you followed the above. "Optional requirements" are not necessary for this project, but the 4 packages mentioned there can be installed via `apt-get` if you are interested. For example,
```
sudo apt-get install libsuitesparse-dev
```
 
### Compilation
First clone the repository, then `cd` into it and follow the 4 steps [there](https://github.com/RainerKuemmerle/g2o#compilation). 

```
mkdir build
cd build
cmake ../
make
```

If you run into any issues, please open an [issue here](https://github.com/Shubodh/MR-project1-pgo/issues).

After the above is successfully done, we want to call it from command-line via `g2o`. To do that, add the following to your `~/.bashrc`:
```
export PATH=$PATH:/your/path/here/g2o/bin/
```
For example,
```
export PATH=$PATH:/home/shubodh/packages/g2o/bin/
```

You can then call `g2o` from within your code via (You can instead even use `g2o_viewer` which allows you to do more than the following command, described in next section):
```
def g2o_call():
    cmd = "g2o -o opt.g2o given.g2o"
    os.system(cmd)
```
`given.g2o` is your given input to g2o optimizer and `opt.g2o` is its output. How to generate `given.g2o`?
The given `edges.txt` is already in g2o format. Just rename it to `edges.g2o`, but it in its current state has only 1 pose (first). You have to generate the rest of the poses using the motion model (see main notebook) and write the rest of the poses just below the first pose. A sample g2o file is as follows:
```
VERTEX_SE2 0 -5.0 -8.0 0.0
VERTEX_SE2 1 -4.55 -8.0 0.0
...
EDGE_SE2 0 1 0.4500000000000002 0.0 0.0 500.0 0.0 0.0 500.0 0.0 500.0
EDGE_SE2 1 2 0.4500000000000002 0.0 0.0 500.0 0.0 0.0 500.0 0.0 500.0
...
FIX 0
EDGE_SE2 0 80 -0.9499999999999993 -0.1999999999999993 0.0 700.0 0.0 0.0 700.0 0.0 700.0
EDGE_SE2 2 82 -0.75 -0.1999999999999993 0.0 700.0 0.0 0.0 700.0 0.0 700.0

```

G2O's format is explained [link here](https://www.notion.so/saishubodh/G2O-Edge-Description-fa07cc28967541dc8a71170de46c5da7).

### g2o viewer

Using this viewer, you can see pose graph along with the loop closure edges. You can also directly optimize instead of using the command described above.

```g2o_viewer file.g2o``` 

Controls: arrow keys/mouse

It has various options which you can play around with before clicking on "Optimize". Don't worry, you are not expected to understand every option available there. But for questions asked, you will have to play around a bit before getting good results. You can see if it is "good results" or not from the image that I inserted in the main notebook.

### g2o read/write helper functions

In [ ]:
def readVertex(fileName):
    f = open(fileName, 'r')
    A = f.readlines()
    f.close()

    x_arr = []
    y_arr = []
    theta_arr = []

    for line in A:
        if "VERTEX_SE2" in line:
            (ver, ind, x, y, theta) = line.split()
            x_arr.append(float(x))
            y_arr.append(float(y))
            theta_arr.append(float(theta.rstrip('\n')))

    return jnp.array([x_arr, y_arr, theta_arr])

def readEdge(fileName):
    f = open(fileName, 'r')
    A = f.readlines()
    f.close()

    ind1_arr = []
    ind2_arr = []
    del_x = []
    del_y = []
    del_theta = []

    for line in A:
        if "EDGE_SE2" in line:
            (edge, ind1, ind2, dx, dy, dtheta, _, _, _, _, _, _) = line.split()
            ind1_arr.append(int(ind1))
            ind2_arr.append(int(ind2))
            del_x.append(float(dx))
            del_y.append(float(dy))
            del_theta.append(float(dtheta))

    return (jnp.array( ind1_arr), jnp.array(ind2_arr), jnp.array(del_x), jnp.array(del_y), jnp.array(del_theta))

## 4. Misc Helper Functions

In [ ]:
# Plotting trajectory using matplotlib
def draw(X, Y, THETA):
    ax = plt.subplot(111)
    ax.plot(X, Y, 'ro')
    plt.plot(X, Y, 'c-')

    for i in range(len(THETA)):
        x2 = 0.25*math.cos(THETA[i]) + X[i]
        y2 = 0.25*math.sin(THETA[i]) + Y[i]
        plt.plot([X[i], x2], [Y[i], y2], 'g->')

    plt.show()

In [ ]:
# Use this to check if your analystic calculation of gradient is correct (compared to jax's `jacfwd`)
def frobNorm(P1, P2, str1="mat1", str2="mat2"):
    jnp.set_printoptions(suppress=True)
    val = jnp.linalg.norm(P1 - P2, 'fro')
    print(f"Frobenius norm between {str1} and {str2} is: {val}")